In [1]:
import numpy as np

Define Activation function

In [2]:
def g(x):
    return np.tanh(x)

In [3]:
def gprime(x):
    return 1.0 - g(x)**2

In [11]:
class NeuralNetwork:
    def __init__(self, L, alpha):

        self.L = L
        self.L[0] += 1
        self.nL = len(self.L)
        self.alpha = alpha

        # initialize weight
        self.W = []
        for l in range(1, self.nL):
            w = np.random.uniform(high=0.25, low=-0.25, size=(self.L[l - 1], self.L[l]))
            self.W.append(w)

        # initialize activity
        self.a = []
        for l in range(0, self.nL):
            tmp = np.ones(self.L[l])
            self.a.append(tmp)

        # initialize z
        self.z = []
        for l in range(0, self.nL):
            tmp = np.zeros(self.L[l])
            self.z.append(tmp)

        # initialize deltas
        self.deltas = []
        for l in range(self.nL-1):
            tmp = np.zeros(self.L[l])
            self.deltas.append(tmp)

    def forward_propagate(self, x):

        x = np.atleast_2d(x)
        tmp = np.ones((x.shape[0], 1))
        x = np.hstack((x, tmp))
        self.a[0] = x

        for l in range(self.nL-1):
            self.z[l+1] = np.dot(self.a[l], self.W[l])
            # call 'g'
            self.a[l+1] = g(self.z[l+1])
        return self.a[-1]
    
    def back_propagate(self, x, y):
        # call 'forward_propagate'
        self.forward_propagate(x)

        error = y - self.a[-1]
        # call 'gprime'
        self.deltas[-1] = np.atleast_2d(error * gprime(self.z[-1]))

        for l in range(self.nL - 2, 0, -1):
            # call 'gprime'
            self.deltas[l-1] = np.dot(self.deltas[l], self.W[l].T)*gprime(self.z[l])

        for l in range(self.nL-1):
            self.W[l] += self.alpha * np.dot(self.a[l].T, self.deltas[l])

    def train(self, x, y, epochs=5000, c=0):
         # check if dimension of x is correct
        if x.shape[1] != self.L[0] - 1:
            raise ValueError('x has wrong dimension')

        for iter in range(1, epochs):

            for s in range(x.shape[0]):
                # call 'back_propagate'
                self.back_propagate(x[s], y[s])

            if c:
                if iter % 100 == 0:
                    # call 'classify'
                    print('iter {}: accuracy {}'.format(iter, self.classify(x, y)))

    def classify(self, x, y):
        correct = 0
        for s in range(x.shape[0]):
            # call 'forward_propagate'
            self.forward_propagate(x[s])
            if self.a[-1] > 0:
                c = 1.0
            else:
                c = -1.0
            if c == y[s]:
                correct += 1
        return correct/x.shape[0]

In [9]:
# call 'NeuralNetwork'
nn = NeuralNetwork([2, 5, 1], 0.01)
data = np.loadtxt('train1.txt')
x = []
y = []
for i in range(data.shape[0]):
    x.append([data[i, 0], data[i, 1]])
    y.append(data[i, 2])
x = np.array(x)
y = np.array(y)
# call 'train'
nn.train(x, y, epochs=1000, c=1)

data = np.loadtxt('test1.txt')
x = []
y = []
for i in range(data.shape[0]):
    x.append([data[i, 0], data[i, 1]])
    y.append(data[i, 2])
x = np.array(x)
y = np.array(y)
# call 'classify'
print('testing accuracy: {}'.format(nn.classify(x, y)))

iter 100: accuracy 0.36
iter 200: accuracy 0.36
iter 300: accuracy 0.68
iter 400: accuracy 0.92
iter 500: accuracy 0.92
iter 600: accuracy 1.0
iter 700: accuracy 1.0
iter 800: accuracy 1.0
iter 900: accuracy 1.0
testing accuracy: 0.96
